In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

"""
Current workflow and organization of code:

Workflow
1. Jingchun has code to generate distance features
2. Harrison has code to generate force features
3. TODO: merge Jinchun and Harrison's dataframes
    a. Reduce memory usage when doing so



"""

In [7]:
# reduce memory usage of each week's dataframe and pickle. In the future, we will only be reading from these pickled files
# alternatively, we also have CSV's

# weeks_dict = dict()
# for i in range(1, 9):
#     headers = [*pd.read_csv(f'week{i}_clean.csv', nrows=1)]
#     weeks_dict[i] = pd.read_csv(f'week{i}_clean.csv', usecols =[i for i in headers if i != 'time'])
#     #reduce memory usage
#     weeks_dict[i]['gameId'] = weeks_dict[i]['gameId'].fillna(0).astype('uint32', errors = 'ignore')
#     weeks_dict[i]['nflId'] = weeks_dict[i]['nflId'].fillna(0).astype('uint32', errors = 'ignore')
#     weeks_dict[i]['playId'] = weeks_dict[i]['playId'].fillna(0).astype('uint16', errors = 'ignore')
#     weeks_dict[i]['frameId'] = weeks_dict[i]['frameId'].fillna(0).astype('uint8', errors = 'ignore')
#     weeks_dict[i]['jerseyNumber'] = weeks_dict[i]['jerseyNumber'].fillna(-1).astype('int8', errors = 'ignore')
#     weeks_dict[i]['team'] = weeks_dict[i]['team'].astype('category', errors = 'ignore')
#     weeks_dict[i]['playDirection'] = weeks_dict[i]['playDirection'].astype('category', errors = 'ignore')
#     weeks_dict[i][weeks_dict[i].select_dtypes(np.float64).columns] = weeks_dict[i].select_dtypes(np.float64).astype(np.float32)
#     weeks_dict[i]['event'] = weeks_dict[i]['event'].astype('category', errors = 'ignore')
#     print(weeks_dict[i].info(memory_usage='deep'))
    
#     # create pickle
#     weeks_dict[i].to_pickle(f'week{i}_clean_pickled.pkl')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843295 entries, 0 to 843294
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   gameId         843295 non-null  uint32  
 1   playId         843295 non-null  uint16  
 2   nflId          843295 non-null  uint32  
 3   jerseyNumber   843295 non-null  int8    
 4   team           843295 non-null  category
 5   playDirection  843295 non-null  category
 6   x              843295 non-null  float32 
 7   y              843295 non-null  float32 
 8   s              843295 non-null  float32 
 9   a              843295 non-null  float32 
 10  dis            843295 non-null  float32 
 11  o              806630 non-null  float32 
 12  dir            806630 non-null  float32 
 13  event          843295 non-null  category
 14  frameId        843295 non-null  uint8   
dtypes: category(3), float32(7), int8(1), uint16(1), uint32(2), uint8(1)
memory usage: 34.6 MB
None
<class 

In [8]:
# read in all the weeks
weeks_dict = dict()
for i in range(1, 9):
    weeks_dict[i] = pd.read_pickle(f'week{i}_clean_pickled.pkl')
    print(weeks_dict[i].info(memory_usage='deep'))
    
# read in rest of data and reduce the memory

games = pd.read_csv("games.csv")
pff = pd.read_csv("pffScoutingData.csv")
pff['gameId'] = pff['gameId'].fillna(-1).astype('uint32', errors = 'ignore')
pff['nflId'] = pff['nflId'].fillna(-1).astype('uint32', errors = 'ignore')
pff['playId'] = pff['playId'].fillna(-1).astype('uint16', errors = 'ignore')
pff['pff_role'] = pff['pff_role'].astype('category')
pff['pff_positionLinedUp'] = pff['pff_positionLinedUp'].astype('category')
pff['pff_hit'] = pff['pff_hit'].fillna(0).astype('int8')
pff['pff_hurry'] = pff['pff_hurry'].fillna(0).astype('int8')
pff['pff_sack'] = pff['pff_sack'].fillna(0).astype('int8')
pff['pff_beatenByDefender'] = pff['pff_beatenByDefender'].fillna(0).astype('int8')
pff['pff_hitAllowed'] = pff['pff_hitAllowed'].fillna(0).astype('int8')
pff['pff_hurryAllowed'] = pff['pff_hurryAllowed'].fillna(0).astype('int8')
pff['pff_sackAllowed'] = pff['pff_sackAllowed'].fillna(0).astype('int8')
pff['pff_nflIdBlockedPlayer'] = pff['pff_nflIdBlockedPlayer'].fillna(0).astype('uint32')
pff['pff_blockType'] = pff['pff_blockType'].astype('category')
pff['pff_backFieldBlock'] = pff['pff_backFieldBlock'].fillna(0).astype('int8')

players = pd.read_csv("players.csv")
play = pd.read_csv("plays.csv")
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843295 entries, 0 to 843294
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   gameId         843295 non-null  uint32  
 1   playId         843295 non-null  uint16  
 2   nflId          843295 non-null  uint32  
 3   jerseyNumber   843295 non-null  int8    
 4   team           843295 non-null  category
 5   playDirection  843295 non-null  category
 6   x              843295 non-null  float32 
 7   y              843295 non-null  float32 
 8   s              843295 non-null  float32 
 9   a              843295 non-null  float32 
 10  dis            843295 non-null  float32 
 11  o              806630 non-null  float32 
 12  dir            806630 non-null  float32 
 13  event          843295 non-null  category
 14  frameId        843295 non-null  uint8   
dtypes: category(3), float32(7), int8(1), uint16(1), uint32(2), uint8(1)
memory usage: 34.6 MB
None
<class 

In [11]:
jingchun_df = pd.read_csv("features_combined.csv")
jingchun_df.drop(columns = ['Unnamed: 0', 'X'], inplace = True)
display(jingchun_df)

,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
0,100,2021091205,1,122.7432,2.168919,4.334588,-231.113662,9.797626,-220.255350,20.433735,...,-144.075354,-18.563033,-166.601470,-9.691129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,100,2021091205,10,114.8473,2.144202,4.286609,-252.178540,55.454604,-248.325300,61.637150,...,-764.944993,41.232356,-730.694702,62.674307,-2185.187496,273.328672,-2320.911151,724.855993,-880.289107,-481.145822
2,100,2021091205,11,178.6333,2.128901,4.282523,-229.912020,58.702866,-227.888060,63.721820,...,-557.301249,39.354358,-534.589494,53.584955,-2416.765871,358.687217,-2223.118612,591.848819,-953.654859,-511.988395
3,100,2021091205,12,115.7391,2.119057,4.276415,-207.652940,46.110528,-203.284560,49.591252,...,-399.025998,39.897900,-387.962010,50.498707,-2268.549479,392.189287,-2047.840067,600.653093,-799.675040,-424.836831
4,100,2021091205,13,124.1564,2.108225,4.269593,-188.890810,37.290250,-183.203288,40.093566,...,-303.236778,27.236087,-297.042415,36.519393,-2002.435484,388.120074,-1901.432203,584.137689,-882.090358,-466.648595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36612,997,2021091201,5,118.1891,2.329580,4.429242,-412.424873,12.711778,-412.949218,20.877553,...,-674.684297,-393.784649,-676.507582,-400.885510,-50.168426,-148.557629,-499.352745,-96.113390,-35.582177,-33.963871
36613,997,2021091201,6,0.0000,2.327283,4.422579,-394.322206,-16.814660,-392.363515,-10.422416,...,-707.334838,-404.018446,-701.911397,-405.589572,-162.532167,-145.883005,-991.138457,-29.333066,-32.144231,-37.828542
36614,997,2021091201,7,118.3133,2.323264,4.399719,-354.162733,-30.644524,-353.870833,-25.974829,...,-724.093765,-366.517346,-714.535755,-371.265664,-243.643664,-143.574383,-1774.600223,-99.234451,-129.630971,-201.040913
36615,997,2021091201,8,918.9437,2.320650,4.363982,-307.997942,-36.733928,-310.530255,-33.920618,...,-620.236820,-291.668549,-605.475788,-296.246401,-910.369328,-264.317115,-2004.284377,-487.593357,-459.737129,-38.415974


In [10]:
jingchun_df.columns

Index(['Unnamed: 0', 'playId', 'gameId', 'frameId', 'area_block', 'dist_block',
       'dist_to_qb_block', 'x_force_block', 'y_force_block',
       'norm_weighted_x_block', 'norm_weighted_y_block', 'area_rush',
       'dist_rush', 'dist_to_qb_rush', 'x_force_rush', 'y_force_rush',
       'norm_weighted_x_rush', 'norm_weighted_y_rush', 'X',
       'net_x_force_bottom', 'net_y_force_bottom', 'net_x_force_middle',
       'net_y_force_y', 'net_x_force_top', 'net_y_force_middle'],
      dtype='object')

In [5]:
partitioned_df = pd.read_csv("partitioned_forces.csv")
partitioned_df = partitioned_df.drop(columns = ['Unnamed: 0'])
partitioned_df

,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
0,2021090900,97,1,92.944313,225.388103,1.455071,2.392231,0.000000,0.000000
1,2021090900,97,2,93.165687,169.537781,1.602810,2.295865,0.000000,0.000000
2,2021090900,97,3,60.928573,111.658176,105.906817,-17.823788,0.000000,0.000000
3,2021090900,97,4,164.840788,17.867436,883.961408,212.598808,42.370021,-32.747270
4,2021090900,97,5,465.906475,-77.330013,761.824479,577.008782,1115.429553,91.194218
...,...,...,...,...,...,...,...,...,...
48609,2021091210,146,43,NaN,NaN,NaN,NaN,-1934.920416,2761.192229
48610,2021091210,146,44,NaN,NaN,NaN,NaN,-2059.174077,2692.567357
48611,2021091210,146,45,NaN,NaN,NaN,NaN,-2141.343067,2622.170778
48612,2021091210,146,46,NaN,NaN,NaN,NaN,-2191.702721,2732.787281


In [14]:
# get data frame by frame
df_grouped = jingchun_df.groupby('frameId')
df_list = []
for n, g in df_grouped:
    if (n % 5 == 1 and n < 60):
        df_list.append(g)
        display(g)

,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
0,100,2021091205,1,122.74320,2.168919,4.334588,-231.113662,9.797626,-220.255350,20.433735,...,-144.075354,-18.563033,-166.601470,-9.691129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
23,100,2021091207,1,142.08250,1.960089,3.965165,140.745802,9.518084,136.701880,6.369264,...,382.655170,36.451731,365.177238,41.823683,-11.409993,42.701898,120.296056,-43.092933,-20.962159,71.177440
50,100,2021091211,1,260.03540,2.419573,4.744737,70.720471,-344.857692,73.254744,-360.416647,...,130.357471,-72.879822,142.116252,-74.330654,21.855746,7.291634,105.708678,-102.562543,NaN,NaN
91,1004,2021091212,1,464.99880,2.024838,4.293123,-314.963790,80.004344,-303.762242,86.175160,...,-330.902410,122.333890,-349.689295,133.687066,0.000000,0.000000,28.689920,-185.127166,-4.450550,-2.234502
127,1013,2021091300,1,72.76843,2.053187,4.905739,-121.813319,54.054223,-111.827303,51.183871,...,-81.237743,34.043428,-79.441067,35.058095,-33.257133,-8.948558,-22.562588,24.219299,-10.912856,31.302268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36491,991,2021091213,1,148.87140,2.159864,2.208332,-72.413184,55.329226,-25.342014,4.731052,...,-80.046999,34.622313,-99.940395,55.535876,NaN,NaN,56.403938,148.017413,0.000000,0.000000
36520,995,2021091209,1,97.73060,2.099159,4.328119,67.581742,37.626654,59.503290,37.695856,...,185.754722,-34.172037,191.395181,-33.147248,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
36538,996,2021091208,1,178.69520,2.068432,4.057318,380.748400,7.212168,370.689160,-1.069798,...,355.914049,-96.553607,364.992095,-110.516442,4.963453,-6.776734,28.649128,-7.919922,-6.888238,28.581675
36569,996,2021091210,1,139.63200,2.012035,4.050893,333.792200,66.117060,325.195280,58.806598,...,419.152328,12.578576,434.152216,23.403770,0.190873,2.743368,0.000000,0.000000,54.905505,55.851083


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
19,100,2021091205,6,79.03103,2.165380,4.273408,-311.417640,45.599314,-303.104700,60.167452,...,-1345.483725,70.183668,-1298.696572,100.161210,-596.385621,-219.190858,-874.767430,311.689433,-116.641368,-99.199417
46,100,2021091207,6,19.18660,1.958878,3.882229,204.114040,-26.939891,203.463134,-30.345867,...,535.394092,-47.080676,513.240365,-32.294788,513.875054,108.477940,1189.433081,127.876617,385.243868,-164.002628
87,100,2021091211,6,0.00000,2.441142,4.757713,67.139128,-696.179939,69.687916,-677.533473,...,492.928413,-481.379288,488.815625,-478.826798,-61.051690,-177.129821,1054.902702,-1868.490821,NaN,NaN
123,1004,2021091212,6,329.58010,2.026799,4.268407,-190.024135,70.757922,-180.034401,76.925696,...,-653.010090,135.524885,-592.142026,132.145014,-740.333545,-59.643827,-1966.018233,906.559233,-192.076870,42.441750
151,1013,2021091300,6,68.69387,2.050543,4.838709,-526.953188,219.635700,-523.032446,224.064460,...,-734.385760,88.798919,-737.150294,93.778621,-142.135445,26.117177,-700.602403,259.661839,-91.279726,120.665853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36516,991,2021091213,6,293.99220,2.180527,2.230467,-399.039506,-126.593880,-315.809246,-42.626730,...,-787.441702,28.075056,-753.355747,86.628881,0.000000,0.000000,-682.253993,415.135424,0.000000,0.000000
36534,995,2021091209,6,42.00570,2.096804,4.324411,235.851644,-92.827980,222.490378,-96.321460,...,606.249161,-7.874444,587.516635,-7.875695,259.632739,-122.903118,440.598268,-4.096663,380.696595,178.444860
36565,996,2021091208,6,146.37900,2.071284,4.008484,610.707740,279.446962,617.488340,280.429604,...,646.815380,-8.999891,626.605285,4.103496,708.113342,-20.150519,2552.081182,-316.232650,67.203677,-13.770472
36586,996,2021091210,6,352.29250,2.038611,4.065821,458.433020,75.818478,464.409740,81.188576,...,694.929970,-17.377300,660.036614,-0.217850,946.834060,-137.161801,1321.613027,57.742126,1077.123186,460.319224


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
2,100,2021091205,11,178.6333,2.128901,4.282523,-229.912020,58.702866,-227.888060,63.721820,...,-557.301249,39.354358,-534.589494,53.584955,-2416.765871,358.687217,-2223.118612,591.848819,-953.654859,-511.988395
25,100,2021091207,11,67.0892,2.025527,3.839529,167.274384,-36.818218,169.131030,-40.978711,...,250.469855,57.098574,269.146520,75.068850,484.982424,-79.158099,1551.575540,-386.769338,1106.405263,-85.004595
52,100,2021091211,11,0.0000,2.449239,4.816018,36.697074,-171.095577,35.953462,-171.182664,...,322.323151,-514.172327,342.149087,-545.445830,871.059201,-785.728251,1555.896901,-4588.217020,NaN,NaN
93,1004,2021091212,11,180.2292,2.106368,4.217441,-179.018804,59.066474,-181.435792,60.530804,...,-290.183276,-9.290994,-295.330428,-7.711116,-1626.897935,382.948572,-1279.972370,684.119640,-655.290730,-171.179051
129,1013,2021091300,11,132.2594,2.049007,4.704041,-260.180540,131.121652,-257.554020,130.876520,...,-277.900720,28.844035,-282.538986,31.175130,-1625.744585,-76.419222,-2734.054358,1179.074028,-1212.510494,350.719413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36493,991,2021091213,11,359.5277,2.356634,2.512313,-528.426380,-95.528348,-472.140600,-45.588632,...,-465.120825,8.860565,-376.944699,26.015267,-832.956507,-616.355392,-2781.035737,-169.545467,-1530.972115,265.231612
36522,995,2021091209,11,250.9121,2.218860,4.432757,324.083920,-71.470400,332.717680,-74.200636,...,234.946596,-10.923880,217.050871,-9.534051,926.807727,-122.573741,1334.114387,-583.987430,1343.332794,210.923511
36540,996,2021091208,11,470.7686,2.363816,3.948040,385.844220,129.659205,387.595000,122.880405,...,409.554757,-43.895484,384.362086,-17.190071,1077.431901,-21.185469,3328.624945,1389.358970,1234.743178,-6.938240
36571,996,2021091210,11,145.2653,2.231462,3.987394,209.929524,30.791783,201.985950,28.559543,...,283.076151,39.681227,281.540549,41.923362,1561.055424,48.302195,2442.594373,587.488478,1068.235180,-326.207427


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
7,100,2021091205,16,97.29533,2.084440,4.290909,-247.612068,63.444758,-249.546164,65.921382,...,-391.786727,6.819328,-376.407445,10.811655,-1038.638060,139.671426,-1330.320335,462.586575,-452.505463,-190.680601
30,100,2021091207,16,208.99370,2.021536,3.908861,277.875066,-77.820799,283.630348,-79.884635,...,401.671205,-7.182550,373.913631,-15.811888,130.807841,-31.139143,985.263932,-253.364478,739.443764,28.657881
57,100,2021091211,16,0.00000,2.292519,4.975004,100.794161,-410.990933,102.135958,-432.402109,...,178.571022,-410.638860,198.415709,-397.713617,462.071456,-862.763842,1018.284545,-2300.145363,NaN,NaN
98,1004,2021091212,16,488.67030,2.259059,4.545952,-276.737480,39.114844,-275.699880,42.022761,...,-618.864263,105.878028,-618.083651,108.009476,-478.711996,64.774925,-1197.222012,185.282145,-379.893102,8.111231
134,1013,2021091300,16,106.64670,2.212953,4.663454,-445.165540,58.751968,-432.265980,61.667570,...,-497.237182,-163.976149,-473.317834,-138.155577,-597.805159,53.373433,-690.060637,418.373724,-1124.639828,299.237250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36498,991,2021091213,16,261.30720,2.704677,3.189400,-387.553380,-96.262620,-372.685000,-88.147354,...,-463.644594,-52.933436,-457.434812,-39.984105,-1297.972464,-551.762666,-1701.956750,97.833323,-1502.685997,11.729870
36527,995,2021091209,16,138.15240,2.631890,4.687397,399.413940,-76.442038,392.108120,-74.278378,...,442.629621,-165.736433,433.963306,-147.078150,419.042481,-53.235367,1285.317222,-330.878978,855.846203,-16.933169
36545,996,2021091208,16,537.05100,2.696556,4.146021,518.855920,179.435556,520.116540,185.427390,...,461.286865,27.635856,467.985565,44.392883,657.903085,105.861667,1877.956625,598.170277,1031.580328,-231.317880
36576,996,2021091210,16,499.03050,2.475223,4.218470,532.477380,195.970070,531.310540,204.621150,...,519.378297,25.412017,522.677095,48.180943,705.532101,162.372527,926.702454,75.646484,549.717692,74.664860


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
13,100,2021091205,21,41.27377,2.128973,4.209363,-300.197130,103.164509,-293.516796,102.291896,...,-610.961786,192.903099,-611.850331,199.306278,-999.751674,181.539667,-721.187824,264.393649,-692.481066,-108.251565
36,100,2021091207,21,63.35360,2.026325,4.084134,285.622540,28.359434,281.077700,25.657464,...,550.742713,187.162073,517.854387,65.535583,742.826207,-22.873158,1369.653552,-620.984314,858.012046,-183.918125
63,100,2021091211,21,89.06890,2.108303,5.783651,135.720481,-588.954970,160.240135,-598.274452,...,355.416420,-467.207887,521.993214,-409.508054,255.013531,-2629.980712,727.379392,-1324.411427,NaN,NaN
104,1004,2021091212,21,223.99940,2.454922,4.956384,-321.218476,53.236925,-323.474494,55.337239,...,-511.201886,167.088898,-506.265516,174.948860,-986.159401,293.385683,-1308.517825,134.811873,-1564.467174,190.888735
140,1013,2021091300,21,65.51617,2.534239,4.794776,-400.017360,73.636158,-394.094980,98.370794,...,-478.079516,-101.949561,-520.094371,-9.523236,-684.173406,84.775155,-1561.018826,-189.625410,-1969.584285,-257.294470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36447,98,2021091206,21,282.38970,2.628896,4.027226,-301.376617,-93.602778,-315.885367,-79.734300,...,-662.701988,388.487451,-691.038149,511.643111,-889.260079,286.878158,-2694.288883,-247.207600,-609.344588,280.784518
36504,991,2021091213,21,154.29110,2.966451,3.778484,-461.136920,-39.794400,-436.281720,-43.145035,...,-579.443708,-170.314674,-562.069647,-163.280197,-1028.652805,-254.485047,-1917.455629,-227.994718,-846.236831,-210.567112
36551,996,2021091208,21,64.75610,2.742819,4.379292,453.195440,64.690812,456.687900,77.057260,...,501.062361,35.041171,510.756717,50.938106,500.447245,203.923267,2109.271867,1034.137830,1829.707984,-230.339912
36582,996,2021091210,21,147.17680,2.470489,4.324498,272.106808,97.436980,269.352726,102.495220,...,422.975854,90.003519,453.887447,70.382719,693.603353,426.040381,3053.097512,791.248441,993.199214,-135.790398


...

,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
79,100,2021091211,36,45.2259,2.530483,14.580962,-229.448636,-139.835345,-223.791741,-86.044070,...,-336.019888,189.111302,-352.332095,395.186009,-1555.108924,-1458.835921,-151.876585,1108.726082,NaN,NaN
120,1004,2021091212,36,422.9794,2.284062,3.910236,-421.535572,-86.841972,-421.451508,-108.322088,...,-111.865964,-543.352459,-121.626954,-588.636251,NaN,NaN,-1554.989396,807.120740,-931.086758,-122.588533
184,1020,2021091210,36,29.8263,2.567875,4.070357,126.556630,249.299542,162.821126,291.425048,...,94.160692,150.760229,63.575863,134.106200,800.551617,-330.722655,533.838848,933.110547,856.988762,719.782590
548,1064,2021091213,36,88.6130,2.495183,3.044012,-380.431800,-341.521960,-376.392620,-366.597980,...,-91.771973,-427.865387,-117.863044,-401.658704,NaN,NaN,-1992.658478,-1028.494999,-527.695151,56.394329
611,1075,2021091300,36,28.8926,1.770247,4.903697,25.170118,37.099100,3.935674,-33.623020,...,65.295872,249.980775,79.370096,270.620431,321.194790,94.471945,185.186265,-494.664180,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36166,947,2021090900,36,15.8479,1.421578,3.508714,402.049402,-106.117316,378.537963,-137.577971,...,436.362234,-18.362175,388.622898,19.619054,890.262073,-83.160705,2144.773522,-1342.414027,NaN,NaN
36253,954,2021091207,36,54.9864,2.415756,3.810689,87.811860,150.193160,96.611480,162.375620,...,171.768553,267.799561,136.189250,355.358808,850.695171,368.740657,948.999227,192.260577,NaN,NaN
36316,962,2021091213,36,118.8793,3.098962,9.475880,-130.570736,208.920714,-148.686808,209.774646,...,-157.666067,646.619103,-159.999258,625.047410,-506.669478,301.676179,-808.130135,1844.948114,-667.072097,1271.412285
36361,966,2021091200,36,214.6504,3.549347,10.342205,6.694086,-228.799831,-3.863398,-229.969362,...,-2.614600,-376.046374,5.555447,-370.735196,-6.062199,-1881.405706,-265.638885,-1392.993953,-153.530557,416.610577


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
85,100,2021091211,41,0.00000,2.731333,17.261649,-146.718677,158.453219,-138.200452,145.368244,...,-303.569875,285.463539,-457.104541,389.669067,-1967.834629,-273.854101,NaN,NaN,-385.059981,676.452986
617,1075,2021091300,41,31.25890,2.312122,5.259917,33.664414,394.339960,35.021592,410.520280,...,201.050795,406.143768,287.382334,447.796299,NaN,NaN,387.034055,1185.418577,NaN,NaN
879,1090,2021091207,41,601.06610,4.899436,9.267560,56.808118,345.770331,57.928734,442.366384,...,382.373661,593.547070,421.916060,651.385598,115.739735,-13.719204,421.582529,3985.599483,-445.370153,578.105114
1053,1122,2021091300,41,234.07650,2.543049,4.502616,415.087483,267.376993,632.308945,262.259960,...,615.882190,-167.344850,497.942921,-346.419061,NaN,NaN,4871.764202,884.724650,NaN,NaN
1328,1153,2021091200,41,303.15200,3.696511,4.541133,263.834109,-119.434704,357.442703,-69.838914,...,160.544241,-101.166311,129.001994,-276.612550,285.957202,-393.647667,2198.986166,-2494.383189,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35081,811,2021091205,41,63.38887,2.904845,3.593233,99.690015,222.197938,137.169705,378.171185,...,86.657791,-175.495444,181.493480,-340.491808,NaN,NaN,1864.326845,3013.002505,NaN,NaN
35279,843,2021091200,41,185.42970,2.393982,2.362640,-98.745898,36.196356,-193.812968,-60.643416,...,-92.746288,34.914964,-134.721369,55.194956,NaN,NaN,1621.861704,-1149.107315,NaN,NaN
35752,902,2021091211,41,11.26570,3.197520,16.027556,130.049570,-87.026600,141.912774,-35.091924,...,96.463856,60.239196,801.308866,-13.372920,739.436984,-2676.635692,557.646578,723.287847,725.772853,21.794019
36322,962,2021091213,41,247.32750,3.428219,11.269168,-29.886916,297.310087,-43.425538,312.368159,...,223.410106,419.345655,129.819060,455.465114,NaN,NaN,-761.982451,2046.257418,-782.676901,2002.663967


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
884,1090,2021091207,46,380.7348,5.287117,10.692360,189.493457,328.111007,272.243161,316.603124,...,381.247616,398.685707,425.587620,412.669638,226.251537,41.076116,955.692312,2795.862950,745.207702,1957.641631
1438,1166,2021091209,46,17.1201,5.368815,8.469781,-101.807188,-146.777509,-89.101773,104.608966,...,-112.598465,-372.737031,-48.008724,-471.903879,2017.781406,812.307917,120.067290,-777.382396,467.137968,-1484.770127
1615,1179,2021091205,46,121.6486,2.227959,9.011259,108.810360,82.053474,109.978394,99.259964,...,241.630686,170.045849,198.395030,218.050509,NaN,NaN,892.403337,-60.123173,508.764123,669.146981
2467,1271,2021091200,46,244.6946,3.149553,10.661202,148.958301,-170.921065,231.965078,-178.826255,...,-81.835892,-362.188596,99.555843,-340.043713,659.926215,-521.911255,-1087.941712,-1577.488529,NaN,NaN
2626,1293,2021091300,46,43.0244,3.022760,5.674947,-197.809582,-379.410952,-237.700696,-349.499362,...,-264.679595,-329.040741,-370.296268,-504.220675,-110.084340,-401.164801,-1002.301639,-2257.961732,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34701,765,2021091203,46,61.3999,2.014341,2.578058,300.365920,337.651700,228.175122,394.934600,...,222.845171,584.485255,271.177408,544.227578,813.353115,195.719262,2962.019058,1923.044712,NaN,NaN
34944,788,2021091212,46,203.5642,4.049868,10.276800,217.993880,26.474540,183.464874,58.837162,...,348.155557,415.801416,306.080726,431.568583,1404.163786,-351.271737,326.565905,-429.414870,-12.689014,188.721963
35086,811,2021091205,46,314.5396,3.542255,3.748804,32.149962,148.299050,51.715497,61.560717,...,509.470935,-332.349280,397.258859,-410.009280,NaN,NaN,1687.667273,-70.565277,-656.238178,526.275598
35284,843,2021091200,46,266.2211,2.873916,3.811515,-284.481974,-77.117608,-296.011232,-57.197108,...,-174.017959,-27.128279,-218.642636,15.683178,NaN,NaN,-817.276205,-387.972230,-47.438760,708.613879


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
890,1090,2021091207,51,258.09890,5.602871,12.962654,211.804526,290.930666,233.294841,289.444139,...,388.293649,359.745749,413.443077,361.512159,138.605470,42.243750,231.204460,1255.999496,2481.634816,2593.276703
1444,1166,2021091209,51,597.04930,4.872376,9.015858,-43.214132,-201.875898,-26.457338,-263.936938,...,60.859948,-239.475688,113.182936,-312.173689,288.979363,-205.568288,-730.415850,-1272.942393,-619.800530,-893.102470
1621,1179,2021091205,51,377.36860,2.664037,11.984557,135.374022,142.908330,136.502568,164.055322,...,316.961212,365.306378,263.649156,416.985836,NaN,NaN,848.982726,2.575842,661.591872,1087.874886
2632,1293,2021091300,51,93.12067,3.290652,5.085771,-321.463260,-142.554294,-387.660360,-297.183182,...,-190.059084,273.193610,-348.087338,261.874447,-474.932018,-998.817524,-1572.834252,-2214.400180,NaN,NaN
2711,1295,2021091209,51,572.71010,3.890048,12.332618,8.792567,188.525512,31.824038,164.661892,...,27.403630,233.850064,147.802107,-213.111670,249.471560,-582.060844,603.773011,333.795919,553.082116,1010.495757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33255,621,2021091207,51,34.27223,4.133627,10.840509,-218.081568,350.515080,-182.202194,347.526180,...,-238.187865,655.267460,-253.804257,542.595681,-725.818561,-208.673626,-1064.512329,878.140528,-177.460984,2587.634445
34207,726,2021091210,51,243.76720,2.923297,4.532248,224.336695,11.765823,227.895327,-158.669655,...,400.462181,-330.054895,1120.224674,-353.397582,NaN,NaN,3225.214892,651.391327,650.751745,1660.115697
34707,765,2021091203,51,56.12960,2.200227,2.369872,235.777336,488.271400,31.816188,377.305798,...,-223.263893,208.217136,-40.076699,52.267269,NaN,NaN,2170.365060,3441.714337,NaN,NaN
34950,788,2021091212,51,829.76370,5.192506,12.621225,166.090505,119.681240,107.704161,169.286960,...,20.949716,719.024136,139.725352,700.719284,1087.822860,-466.448830,951.798453,873.395705,94.814720,972.830121


,playId,gameId,frameId,area_block,dist_block,dist_to_qb_block,x_force_block,y_force_block,norm_weighted_x_block,norm_weighted_y_block,...,x_force_rush,y_force_rush,norm_weighted_x_rush,norm_weighted_y_rush,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_y,net_x_force_top,net_y_force_middle
895,1090,2021091207,56,784.638600,5.870719,15.868974,217.848539,262.943100,236.739939,275.341461,...,332.588850,420.267403,352.971709,430.627460,NaN,NaN,320.469874,881.838475,2715.336346,2593.659232
1449,1166,2021091209,56,111.561400,4.491062,10.811026,0.864223,-176.960553,28.511500,-222.211707,...,73.952864,-373.103960,124.374319,-362.419197,980.044897,-1123.571632,-624.864628,-581.815100,-371.025242,-463.771372
1626,1179,2021091205,56,501.190600,3.113980,15.093780,139.172394,195.603749,139.571232,219.670870,...,241.317710,517.500163,217.936280,583.463722,NaN,NaN,992.706589,232.611700,952.008487,1943.155450
2637,1293,2021091300,56,35.012730,3.487365,4.735133,-391.392793,344.096982,-485.666646,318.050460,...,70.594986,468.606667,33.516659,465.101337,111.476657,526.771905,-2479.029337,-146.769113,NaN,NaN
2716,1295,2021091209,56,390.054200,5.036272,14.040778,-416.486717,-173.650833,-397.391350,-210.129417,...,-548.672988,-383.281167,-432.454427,-446.584199,53.155631,-737.948024,-810.176865,-650.547230,919.391132,3455.048659
3753,1419,2021091202,56,640.529100,4.567716,23.296278,-48.086266,-163.687694,-36.601911,-173.037040,...,-74.264325,-470.151541,-11.939391,-468.713752,-123.806583,-1604.393730,41.482764,-793.676187,-468.607540,-424.480280
3905,1425,2021090900,56,880.797100,7.285712,18.923803,-59.320899,-355.563505,-94.332166,-374.562291,...,-310.663468,-533.690781,-329.991855,-560.415001,-718.286306,-1662.612001,-222.517449,-1066.420552,395.877297,-502.311003
5088,1521,2021091203,56,502.120500,4.431671,6.393343,166.977459,208.684598,289.444900,291.605096,...,252.312349,398.978688,470.638650,192.947164,1725.045722,416.751523,192.072684,1028.833845,1057.796376,679.432182
7090,1739,2021091205,56,30.299670,3.772152,11.327801,195.764450,-373.555660,190.753860,-397.721916,...,124.508983,-405.240924,179.051411,-409.167499,196.660569,-1683.868205,195.726928,-696.531209,192.791825,-657.138441
7263,1747,2021091207,56,67.241500,6.510890,7.008048,422.848556,-145.809520,529.688832,-147.402884,...,574.150178,738.263272,568.745181,785.472251,3743.285063,-4638.913412,586.876190,-1846.871247,NaN,NaN
